# By Jimin, to understand the codes of HADAR

Started 2023-08-26

In [1]:
import numpy as np
import scipy.io as scio
import os

import torch.nn.functional as F
import torch

/home/ljm/anaconda3/envs/texnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = '~/workspace/zx/HADAR_database/'

## Dataset

The directory names of the downloaded data are renamed to Scene1, Scene2, ..., Scene10.

`matName_FullDatabase.mat`: the materials in the dataset, 30 kinds in total. 

sky asphalt tilePavement cinderblock brick wall marble stone soil al weatheredMetal brass al2o3 oxidizedSteel iron

carpaint plasticPaint yellowSpray carwindow crystalGlass tire cloths card tree grass flower water human bark concrete

In [7]:
materialsFile = os.path.join(root, 'matName_FullDatabase.mat')
materials = np.squeeze(scio.loadmat(materialsFile)['matName'])
materialsName = list()
for i in range(materials.shape[0]):
    materialsName.append(materials[i][0])
print(materialsName)

['sky', 'asphalt', 'tilePavement', 'cinderblock', 'brick', 'wall', 'marble', 'stone', 'soil', 'al', 'weatheredMetal', 'brass', 'al2o3', 'oxidizedSteel', 'iron', 'carpaint', 'plasticPaint', 'yellowSpray', 'carwindow', 'crystalGlass', 'tire', 'cloths', 'card', 'tree', 'grass', 'flower', 'water', 'human', 'bark', 'concrete']


## Preprocessing the data of scene 1 to 10. The data of scence 11 are different. 

`/groundtruth/eMap/eList.mat`: emissivity or index of the materials (not sure) contained in the scence, values in 1-30. It can be seen as the global index of materials.

`/groundtruth/eMap/eMap_L(R)_000X.mat`: index of the materials in eList, values in 1-len(eList). It can be seen as the local scene index. 

eList preprocessing change the local index to global index. The results are save as 
`/groundtruth/eMap/new_eMap_L(R)_000X.npy`

### eMap, material index map, change from Matlab's mat to Numpy's npy.

In [9]:
################ eList Processing #######################
ids = [f"L_{i:04d}" for i in range(1, 6)]
ids += [f"R_{i:04d}" for i in range(1, 6)]
SUBFOLDERS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
SUBFOLDERS = ["Scene"+str(_) for _ in SUBFOLDERS]

print("eMap preprocessing")
for subfolder in SUBFOLDERS:
    print("Processing subfolder", subfolder)
    elist_file = os.path.join(root, subfolder, 'GroundTruth', 'eMap', f"eList.mat")
    e_list = np.squeeze(np.asarray(scio.loadmat(elist_file)["eList"]))
    print(e_list.shape[0], "materials included: ", np.array(materialsName)[e_list-1], "\n")

    e_files = []
    for id in ids:
        e_files.append(os.path.join(root, subfolder, 'GroundTruth', 'eMap', f"eMap_{id}.mat"))

    for i in range(10):
        e_data = scio.loadmat(e_files[i])["eMap"]

        data = ((e_list[e_data - 1] - 1))
        if i<5:
            np.save(os.path.join(root, subfolder, "GroundTruth", "eMap", "new_eMap_L_000"+str(i+1)+".npy"),
                    np.asarray(data))
        else:
            np.save(os.path.join(root, subfolder, "GroundTruth", "eMap", "new_eMap_R_000"+str(i-4)+".npy"),
                    np.asarray(data))

eMap preprocessing
Processing subfolder Scene1
20 materials included:  ['cloths' 'human' 'carwindow' 'crystalGlass' 'carpaint' 'al'
 'weatheredMetal' 'tire' 'asphalt' 'tilePavement' 'brass' 'al2o3'
 'plasticPaint' 'yellowSpray' 'sky' 'brick' 'oxidizedSteel' 'cinderblock'
 'marble' 'iron'] 

Processing subfolder Scene2
20 materials included:  ['cloths' 'human' 'carwindow' 'crystalGlass' 'carpaint' 'al'
 'weatheredMetal' 'tire' 'asphalt' 'tilePavement' 'brass' 'al2o3'
 'plasticPaint' 'yellowSpray' 'sky' 'brick' 'oxidizedSteel' 'cinderblock'
 'marble' 'iron'] 

Processing subfolder Scene3
10 materials included:  ['sky' 'asphalt' 'tilePavement' 'stone' 'grass' 'tree' 'card'
 'weatheredMetal' 'carwindow' 'wall'] 

Processing subfolder Scene4
4 materials included:  ['sky' 'soil' 'grass' 'tree'] 

Processing subfolder Scene5
9 materials included:  ['sky' 'asphalt' 'cinderblock' 'soil' 'grass' 'water' 'al' 'carpaint'
 'carwindow'] 

Processing subfolder Scene6
7 materials included:  ['sky' 'tr

Scene11 is monocular and only has 4 frames.

The data file name of heatcubes in `\Scene11\HeatCubes` should renamed from `heatcube_000X.mat` to `000X_heatcube.mat` to be consistent with other scenes.

In [8]:
ids = [f"{i:04d}" for i in range(1, 5)]
# ids += [f"R_{i:04d}" for i in range(1, 5)]
SUBFOLDERS = [11]
SUBFOLDERS = ["Scene"+str(_) for _ in SUBFOLDERS]
for subfolder in SUBFOLDERS:
    print("Processing subfolder", subfolder)
    e_files = []
    elist_file = os.path.join(root, subfolder, 'GroundTruth', 'eMap', f"eList.mat")
    e_list = np.squeeze(np.asarray(scio.loadmat(elist_file)["eList"]))
    print(e_list.shape[0], "materials included: ", np.array(materialsName)[e_list-1], "\n")
    
    for id in ids:
        e_files.append(os.path.join(root, subfolder, 'GroundTruth', 'eMap', f"eMap_{id}.mat"))

    for i in range(4):
        e_data = scio.loadmat(e_files[i])["eMap"]

        data = ((e_list[e_data - 1]-1))
        np.save(os.path.join(root, subfolder, "GroundTruth", "eMap", "new_eMap_000"+str(i+1)+".npy"), np.asarray(data))

Processing subfolder Scene11
6 materials included:  ['soil' 'grass' 'bark' 'water' 'sky' 'concrete'] 




### S_EnvObj_000X.npy, mean of the multispectral heatcubes, S_beta

For scene 1-10, generate S_EnvObj_L/R_000X.npy from the heatcubes. 

Heatcube: 1080x1920x54 ---> 1x54x1080x1920 --AvgPooling--> 1x54x2x1. Useless operation, squeezed laterly. 

For scene 11, loaded from mat. 1x49x2x1. 

The heatcube is divided into top (cloudy sky) and bottom (ground) halfs and then calculate their mean respectively. 

See the Equation S48 in Nature paper's SI. 

In [12]:
################### S_beta Processing ###################################################

# Separate for Scene 11
S_files = [os.path.join(root, "Scene11", "Radiance_EnvObj", f"S_EnvObj_{i:04d}.mat") for i in range(1, 5)]

for j in range(4):
    data = scio.loadmat(S_files[j])["S_EnvObj"] # 2x49  两个显著环境物，天空和地面，作为S_beta
    data = np.transpose(data)[np.newaxis, ..., np.newaxis] # 1x49x2x1
    np.save(os.path.join(root, "Scene11", "HeatCubes", f"S_EnvObj_{j+1:04d}.npy"), data)

In [4]:
SUBFOLDERS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
SUBFOLDERS = ["Scene"+str(_) for _ in SUBFOLDERS]
ids = [f"L_{i:04d}" for i in range(1, 6)]
ids += [f"R_{i:04d}" for i in range(1, 6)]

print("S_beta preprocessing")
for subfolder in SUBFOLDERS:
    S_files = []
    print("processing subfolder", subfolder)
    for id in ids:
        S_files.append(os.path.join(root, subfolder, 'HeatCubes', f"{id}_heatcube.mat"))

    for j in range(10):
        img = torch.tensor(np.asarray(scio.loadmat(S_files[j])["S"]))   # torch.Size([1080, 1920, 54])
        img = torch.permute(img, (2, 0, 1))                             # torch.Size([54, 1080, 1920])
        img = torch.reshape(img, (1, 54, 1080, 1920))                   # torch.Size([1, 54, 1080, 1920])
        [b, c, h, w] = img.shape
        quadratic_split = F.avg_pool2d(img, (h//2, w))  # 平均池化，将图像沿着高度方向切割成了两部分，并对每个部分计算了平均值
        mean = quadratic_split.numpy()                  # 得到来自天空和地面的两个辐射S_beta，  1x54x2x1

        if j < 5:
            np.save(os.path.join(root, subfolder, "HeatCubes", "S_EnvObj_L_000"+str(j+1)+".npy"), np.asarray(mean))
        else:
            np.save(os.path.join(root, subfolder, "HeatCubes", "S_EnvObj_R_000"+str(j-4)+".npy"), np.asarray(mean))

S_beta preprocessing
processing subfolder Scene1
processing subfolder Scene2
processing subfolder Scene3
processing subfolder Scene4
processing subfolder Scene5
processing subfolder Scene6
processing subfolder Scene7
processing subfolder Scene8
processing subfolder Scene9
processing subfolder Scene10
